In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata

from torchvision import transforms, datasets

In [2]:
''' 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 2.0.0+cu118  Device: cuda


In [3]:
# BATCH_SIZE -> 1개의 Mini-Batch 단위에 대해서 구성된 데이터의 갯수
# EPOCHS -> 존재하고 있는 Mini-Batch를 전부 이용해서 학습한 횟수

# Iteration -> 1개의 Mini-Batch를 통해 학습한 횟수

# 예를 들어, 전체 데이터가 1만개, Batch Size가 1000개일 때,
# 학습은 1 Epoch당 Iteration이 10회 발생함

BATCH_SIZE = 32
EPOCHS = 10

In [4]:
train_dataset = datasets.CIFAR10(
    root = "../data/CIFAR_10", # 저장할 디렉토리 지정
    train = True, # Train 데이터셋인지 Test 데이터셋인지 지정
    download = True, # 해당 데이터를 인터넷에서 다운로드해 이용할 것인지 지정
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(), # 이미지를 50% 확률로 좌우반전
        transforms.ToTensor(),  # 0~255값으로 이뤄진 픽셀값을 0~1로 변환하는 작업
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Tensor로 변화한 이미지에 정규화를 진행
    ])
)

test_dataset = datasets.CIFAR10(
    root = "../data/CIFAR_10",
    train = False,
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(), # 이미지를 50% 확률로 좌우반전
        transforms.ToTensor(),  # 0~255값으로 이뤄진 픽셀값을 0~1로 변환하는 작업
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Tensor로 변화한 이미지에 정규화를 진행
    ])
)

# 데이터를 iterator 객체로 변환시켜주는 함수
train_loader = torchdata.DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_loader = torchdata.DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False
)

100%|██████████| 170498071/170498071 [00:02<00:00, 59346134.38it/s]


Extracting ../data/CIFAR_10/cifar-10-python.tar.gz to ../data/CIFAR_10


In [5]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(planes))
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride = 1)
        self.layer2 = self._make_layer(32, 2, stride = 2)
        self.layer3 = self._make_layer(64, 2, stride = 2)
        self.linear = nn.Linear(64, num_classes)
        
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks  - 1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [6]:
''' 7. Optimizer, Objective Function 설정하기 '''
model = ResNet().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [7]:
''' 8. ResNet 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [8]:
''' 9. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [9]:
''' 10. ResNet 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.339352
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.395791
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.541912
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.465015
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 0.989608
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.122511
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.118723
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.183626

[EPOCH: 1], 	Test Loss: 1.0034, 	Test Accuracy: 63.85 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 0.975028
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 0.747065
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 0.830726
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 0.924542
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 0.814381
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 0.946669
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 0.848464
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 1.099561

[EPOCH: 2], 	Test Loss: 0.7875, 	Test Accuracy: 72.47 % 